In [10]:
import shopify
import os
from dotenv import load_dotenv
import requests
import logging
from bs4 import BeautifulSoup


In [11]:
shop_url = "https://429eef-90.myshopify.com/"
api_version = '2024-01'
# state = binascii.b2a_hex(os.urandom(15)).decode("utf-8")
# redirect_uri = "http://myapp.com/auth/shopify/callback"
# scopes = ['read_apps', 'write_files', 'read_files', 'write_products', 'read_products', 'read_content', 'write_content', 'write_product_feeds', 'read_product_feeds', 'write_product_listings', 'read_product_listings']



load_dotenv()
access_token = (os.getenv('access_token'))

session = shopify.Session(shop_url, api_version, access_token)
shopify.ShopifyResource.activate_session(session)

In [3]:
def get_search_results(url):
    """
    Получает результаты поиска для заданного URL.
    """
    try:
        with requests.get(url) as response:
            if response.status_code == 200:
                return response.text
            else:
                logging.error("Ошибка при получении страницы с результатами поиска: %d", response.status_code)
                return None
    except requests.RequestException as e:
        logging.error("Ошибка при запросе страницы: %s", str(e))
        return None


In [4]:
def find_prod(id):
        # Find product by product_id
        # id = "8764450046282"
        # sku = "941144"
        # title = "Gel Polish - Adventure, 10ml"
        products = shopify.Product.find(handle=id)  # Replace "Your Product ID" with the actual product_id
        for product in products:
            if product:
                # variants = product.attributes['variants']
                return product.attributes
                print()
            else:
                print("Product not found")
                return None



In [5]:
def find_varian(id):
        # Find product by product_id
        # id = "8764450046282"
        sku = "941144"
        # title = "Gel Polish - Adventure, 10ml"
        products = shopify.V .find(sku=sku)  # Replace "Your Product ID" with the actual product_id
        for product in products:
            if product:
                # variants = product.attributes['variants']
                return product.attributes
                print()
            else:
                print("Product not found")
                return None



In [6]:
def write_to_file(text):
    """Записывает строку в указанный файл.

    Args:
        file_name (str): Имя файла, в который нужно записать текст.
        text (str): Текст для записи в файл.
    """
    # Открываем файл для записи (если файл не существует, он будет создан)
    with open('skuNotFind.txt', 'a') as file:  # Используем 'a' для добавления текста в конец файла
        # Добавляем строку с новой строкой в конце
        file.write(text + '\n')

In [12]:
len(shopify.Variant.find(limit=250))


250

In [13]:
for product in shopify.Product.find(limit=250):

    print(product.title)

Bottle Gel Clear with brush, 15ml
Bottle Gel Cover with brush, 15ml
Bottle Gel Milky Pink with brush, 15ml
Bottle Gel Milky White Intense with brush, 15ml
cat’s eye gel polish magnet, neodymium, double-sided
Gel Lak Gel Polish - Throne, 10 Ml
Gel Polish - Addicted, 10ml
Gel Polish - Adventure, 10ml
Gel Polish - Alfa, 10ml
Gel Polish - Almandine, 10ml
Gel Polish - Amazonia, 10ml
Gel Polish - Amethyst, 10ml
Gel Polish - Amour, 10ml
Gel Polish - Ananas, 10ml
Gel Polish - Anonymous,  10ml
Gel Polish - Aperol Spritz, 10ml
Gel Polish - Aphrodisiac, 10ml
Gel Polish - Apple Pie, 10ml
Gel Polish - Appletini, 10ml
Gel Polish - Ariel, 10ml
Gel Polish - Avatar, 10ml
Gel Polish - Aventurine, 10ml
Gel Polish - Baby Blue, 10ml
Gel Polish - Baby Girl, 10ml
Gel Polish - Bad Boys, 10ml
Gel Polish - Baklava, 10ml
Gel Polish - Banana Mama, 10ml
Gel Polish - Banana Shake, 10ml
Gel Polish - Base - New, 10ml
Gel Polish - Bauble, 10ml
Gel Polish - Be Wild, 10ml
Gel Polish - Beatrice, 10ml
Gel Polish - Beautif

In [ ]:
def get_all_variants(limit=250):
    """Fetches all variants from Shopify, handling pagination."""
    all_variants = []

    # Initial request
    variants = shopify.Variant.find(limit=limit)

    # Check if it's a PaginatedCollection (multiple pages)
    

        # Iterate through subsequent pages (if any)
    while variants.has_next_page():
        variants = variants.next_page()
        all_variants.extend(variants)
    
        # Handle case where there's only one page (less than 250 variants)
     

    return all_variants

all_variants = get_all_variants()
print('Total variants:', len(all_variants))
print()
for variant in all_variants:
    sku = variant.sku
    print(sku)

    if not variant.barcode:
        url = 'https://www.tufishop.de/en/?subcats=Y&pcode_from_q=Y&pname=Y&search_performed=Y&q='+str(sku)+'&dispatch=products.search&_gl=1*1o43pxb*_up*MQ..*_ga*MTY3MTAzNzE3Mi4xNzEzODEyNzUw*_ga_Q8841MRH6K*MTcxMzgxMjc0OS4xLjAuMTcxMzgxMjc0OS4wLjAuMA..'

        html = get_search_results(url)
        if html:
            soup = BeautifulSoup(html, 'html.parser')
            links = soup.find_all('a', class_='product-title')
            if not links:
                print('no link:'+sku)
                write_to_file(sku)
                print()
            else:
                link = links[0]['href']
                html = get_search_results(link)
                if html:
                    soup = BeautifulSoup(html, 'html.parser')
                    ean_label = soup.find('label', text='EAN:')

                    if ean_label:
                        ean = ean_label.find_next_sibling('span').get_text(strip=True)

                        print(ean)
                        variant.barcode = ean
                        variant.save()
                        print('barcode good')
                        print()
    else:
        print('barcode exist')
        print()


In [2]:
def findEANsku(sku):

    print(sku)

    if not variant.barcode:
        url = 'https://www.tufishop.de/en/?subcats=Y&pcode_from_q=Y&pname=Y&search_performed=Y&q='+str(sku)+'&dispatch=products.search&_gl=1*1o43pxb*_up*MQ..*_ga*MTY3MTAzNzE3Mi4xNzEzODEyNzUw*_ga_Q8841MRH6K*MTcxMzgxMjc0OS4xLjAuMTcxMzgxMjc0OS4wLjAuMA..'

        html = get_search_results(url)
        if html:
            soup = BeautifulSoup(html, 'html.parser')
            links = soup.find_all('a', class_='product-title')
            if not links:
                print('no link:'+sku)
                write_to_file(sku)
                print()
            else:
                link = links[0]['href']
                html = get_search_results(link)
                if html:
                    soup = BeautifulSoup(html, 'html.parser')
                    ean_label = soup.find('label', text='EAN:')

                    if ean_label:
                        ean = ean_label.find_next_sibling('span').get_text(strip=True)

                        print(ean)
                        variant.barcode = ean
                        variant.save()
                        print('barcode good')
                        print()
    else:
        print('barcode exist')
        print()


In [17]:
shopify.Variant.find(product_id="8773956272458")[0]

variant(47936474480970)

In [4]:
findEANsku('942954')

942954


NameError: name 'variant' is not defined

In [ ]:
find_prod("gel-polish-la-plena-10ml").


In [ ]:
shopify.Product.find(8764450046282).attributes

In [ ]:
shopify.Variant.find(47903988121930).attributes

In [ ]:
def create_variant(product_id, variant_data):
    """
    Creates a new variant for a product on Shopify using the provided data.
    """
    variant = shopify.Variant()
    variant.product_id = product_id
    variant.title = variant_data.get('title', 'Default Title')
    variant.price = variant_data.get('price', '0.00')
    variant.sku = variant_data.get('sku', '')
    variant.compare_at_price = variant_data.get('compare_at_price', '')
    variant.grams = variant_data.get('grams', 0)
    variant.weight = variant_data.get('weight', 0)
    variant.weight_unit = variant_data.get('weight_unit', 'kg')

    # Save the variant
    if variant.save():
        print("Variant created successfully!")
        return variant



In [ ]:
product_id = 8766703010122  # Replace with the actual product ID
variant_data = {
    'title': 'Red',
    'price': '20.00',
    'sku': 'SKU123',
    'grams': 100,
    'weight': 0.1,
    'weight_unit': 'kg'
}

created_variant = create_variant(product_id, variant_data)


In [ ]:
shopify.Product.find(8764449456458).attributes

In [15]:
shopify.Variant.find(limit=250, page=2
                     )

BadRequest: Response(code=400, body="b'{"errors":{"page":"page cannot be passed. See https:\\/\\/shopify.dev\\/api\\/usage\\/pagination-rest for more information."}}'", headers={'Date': 'Mon, 29 Apr 2024 18:32:12 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'close', 'X-Sorting-Hat-PodId': '329', 'X-Sorting-Hat-ShopId': '81976951114', 'referrer-policy': 'origin-when-cross-origin', 'x-frame-options': 'DENY', 'x-shopid': '81976951114', 'x-shardid': '329', 'x-stats-userid': '', 'x-stats-apiclientid': '108093177857', 'x-stats-apipermissionid': '618398351690', 'x-shopify-api-version': '2024-01', 'http_x_shopify_shop_api_call_limit': '1/40', 'x-shopify-shop-api-call-limit': '1/40', 'x-shopify-api-deprecated-reason': 'https://shopify.dev/api/admin-rest/latest/resources/product-variant', 'strict-transport-security': 'max-age=7889238', 'x-request-id': 'e90d6b11-301b-4072-bcdd-b05600b2f57b-1714415532', 'server-timing': 'processing;dur=28',